## Imports

In [3]:
import scipy.io as sio
import numpy as np
import sklearn
from matplotlib import pyplot as plt
from math import e
from math import log
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sys import maxsize
from scipy import stats
from sklearn.tree import DecisionTreeClassifier, export_graphviz

## Load the Data

In [5]:
# Load the data here

## Code:

### Random Forest

In [6]:
class RandomForest:
    def __init__(self, size=40, max_depth = 30, trees=[]):
        self.size = size
        self.max_depth = max_depth
        self.trees = []
    
    def train(self, data, labels):
        trees = []
        for i in range(self.size):
            sub_S = []
            sub_L = []
            for i in np.random.choice(len(labels), len(labels)):
                sub_S.append(data[i])
                sub_L.append(labels[i])
            sub_S = np.array(sub_S)
            sub_L = np.array(sub_L)
            clf = DecisionTree(max_depth=self.max_depth, from_tree = True)
            clf.train(sub_S, sub_L)
            self.trees.append(clf)
    def predict(self, X):
        p = []
        for t in self.trees:
            p.append(t.predict(X))
        p = np.array(p)
        if p.ndim == 1:
            p = p.reshape((len(p), 1))
        best = []
        for i in range(p.shape[1]):
            best.append(stats.mode(p[:,i])[0][0])
        return best

## Visualizing

In [7]:
# Visualize data here

___________________________________________________________________________________________________________________
## \#\#\#FEEL FREE TO IGNORE\#\#\#.
## CODE BELOW FOR DECISION TREE CLASS

### Node Class

In [8]:
class Node():
    def __init__(self, split_rule=[0, 0.5], label=None, _type=None, left=None, right=None, height = 0):
        self.label = label # mode of the labels of the data_points here
        self.split_rule = split_rule
        self.left = left
        self.right = right
        self._type = None
        self.height = height
        if label != None:
            self._type  = "leaf"

###    DECISION TREE :

In [9]:
class DecisionTree():
    def __init__(self, node = None, max_depth = 40, from_tree = False):
        self.node = Node(node)
        self.max_depth = max_depth
        self.from_tree = from_tree
    from operator import itemgetter
    def max_val(l, i): # assuming this shit works
        return max(enumerate(map(itemgetter(i), l)),key=itemgetter(1))

    def train(self, data, labels):
        self.node = self.Grow(data, labels, 0)

    def Grow(self, data, labels, counter):
        d = all_done(labels)  # = [Boolean-if-done, Class-to-choose]
        if d[0] == True:
            r = d
            return Node(label=r[1], height=self.node.height+1)
        if counter >= self.max_depth:
            return Node(label=stats.mode(labels)[0][0], height=self.node.height+1)
        else:
            feat = self.segmenter(data, labels) # feat is the index of the feature
            if feat == False:
                return Node(label=stats.mode(labels)[0][0], height=self.node.height+1)
            split_val = feat[1] # split_val is the value to split on
            feat = feat[0]
            dd = np.append(data, labels.reshape((len(labels),1)), 1)
            dd = dd[dd[:,feat].argsort()]
            if split_val == 0.5 or split_val == 0:
                if len(dd[:,feat]) <= 2:
                    sp = 1
                else:
                    sp = 1
            else:
                sp = dd[:,feat].tolist().index(split_val)
            Sl = [dd[x] for x in range(sp)]
            Sr = [dd[x] for x in range(sp, len(dd))]
            return Node(split_rule=[feat,split_val],
                        left=self.Grow(np.array(Sl)[:,:-1],np.array(Sl)[:,-1], counter+1),
                        right=self.Grow(np.array(Sr)[:,:-1],np.array(Sr)[:,-1], counter+1), height=self.node.height+1)

    def predict(self, data):
        j = []
        if type(data[0]) == np.ndarray:
            for i in data:
                n = self.node
                while n._type is None:
                    rule_f = n.split_rule[0]
                    rule_v = n.split_rule[1]
                    if i[rule_f] < rule_v:
                        n = n.left
                    else:
                        n = n.right
                j.append(n.label)
            return j
        else:
            n = self.node
            while n._type is None:
                    rule_f = n.split_rule[0]
                    rule_v = n.split_rule[1]
                    print("feat = ", rule_f, " , ", "split_val = ", rule_v)
                    if data[rule_f] < rule_v:
                        print("LEFT")
                        n = n.left
                    else:
                        print("RIGHT")
                        n = n.right
        return n.label
    
    def segmenter(self, data, labels):
        off, best_feature, best_in_feature, num_features = [], [], [], data.shape[1]
        same = 0
        t = [i for i in range(len(data))]
        if self.from_tree == True:
            off = np.random.choice((data.shape[1]-1), data.shape[1]-int(np.sqrt((data.shape[1]-1))) , replace=False)
        ent = entropy(t, labels)
        for i in range(num_features): # features
            if i in off:
                continue
            features = _get_features(data[:,i])
            hist = []
            if len(features) <= 2:
                for f in features:
                    for j in range(len(data)):
                        if data[j][i] == f:
                            features[f].append(j)
                    hist.append(features[f])
                if len(features) <= 1:
                    same+=1
                    best_feature.append((i, 0.5, ent - ent))
                else:
                    best_feature.append((i, 0.5, ent - impurity(hist[0], hist[1], labels)))
            else:
                _xs = len([u for u in labels if u == 0])
                _os = len(labels) - _xs
                fs, best_in_feature, best_in_feature = [], [], np.append(data, labels.reshape((len(labels),1)), 1)
                dd = dd[dd[:,i].argsort()]
                old = dd[0][i]
                count, __xs, __os = 0
                fs.append(old)
                for pt in dd:
                    if old == pt[i]:
                        if pt[-1] == 0:
                            _xs -= 1
                            __xs += 1
                        else:
                            _os -= 1
                            __os += 1
                    else:
                        best_in_feature.append(ent - man_weighted_av_entr(__xs,__os,_xs,_os))
                        fs.append(pt[i])
                        if pt[-1] == 0:
                            _xs -= 1
                            __xs += 1
                        else:
                            _os -= 1
                            __os += 1
                ind = best_in_feature.index(max(best_in_feature))
                if ind + 1 != len(best_in_feature):
                    ind += 1
                    best_feature.append((i, fs[ind], best_in_feature[ind - 1]))
                else:
                    best_feature.append((i, fs[ind], best_in_feature[ind]))
        if same == num_features-1:
            return False
        m = max_val(best_feature, -1)
        return (best_feature[m[0]][0], best_feature[m[0]][1])